In [2]:
from tensorflow.keras import layers, models
from matplotlib import pyplot as plt
import nibabel as nib
import numpy as np
import glob

In [3]:
from tensorflow.keras import layers, models
from matplotlib import pyplot as plt
import numpy as np

class AutoEncoder3D:
    def __init__(self, input_shape):
        # Encoder
        encoder_input = layers.Input(shape=input_shape)
        x = layers.Conv3D(32, (3, 3, 3), activation="relu", padding="same")(encoder_input)
        x = layers.MaxPooling3D((2, 2, 2), padding="same")(x)
        x = layers.Conv3D(16, (3, 3, 3), activation="relu", padding="same")(x)
        x = layers.MaxPooling3D((2, 2, 2), padding="same")(x)
        x = layers.Conv3D(8, (3, 3, 3), activation="relu", padding="same")(x)
        encoded = layers.MaxPooling3D((2, 2, 2), padding="same")(x)

        # Decoder
        x = layers.Conv3D(8, (3, 3, 3), activation="relu", padding="same")(encoded)
        x = layers.UpSampling3D((2, 2, 2))(x)
        x = layers.Conv3D(16, (3, 3, 3), activation="relu", padding="same")(x)
        x = layers.UpSampling3D((2, 2, 2))(x)
        x = layers.Conv3D(32, (3, 3, 3), activation="relu", padding="same")(x)
        x = layers.UpSampling3D((2, 2, 2))(x)
        decoded = layers.Conv3D(input_shape[-1], (3, 3, 3), activation="sigmoid", padding="same")(x)

        # Full Autoencoder
        self.autoencoder = models.Model(encoder_input, decoded)
        self.encoder = models.Model(encoder_input, encoded)

        self.autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

    def train(self, x_train, epochs=50, batch_size=16):
        history = self.autoencoder.fit(
            x_train, x_train,
            epochs=epochs,
            batch_size=batch_size,
            shuffle=True,
            validation_split=0.2
        )
        return history


In [27]:
scan_paths = glob.glob("../../Data/Temp/ADNI/**/*.nii", recursive=True)
scans = []
for path in scan_paths:
    scans.append(nib.load(path).get_fdata())
scans = np.array(scans)
scans.shape

(25, 256, 256, 170)

In [32]:
scans_ = np.expand_dims(scans, axis=-1) 
scans_.shape

(25, 256, 256, 170, 1)

In [ ]:
autoencoder = AutoEncoder3D(scans_.shape[1:])
history = autoencoder.train(scans, epochs=2, batch_size=8)

# Plot the training loss
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.legend()
plt.show()


Epoch 1/2


ValueError: Arguments `target` and `output` must have the same shape. Received: target.shape=(None, 256, 256, 170, 1), output.shape=(None, 256, 256, 176, 1)